In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys 
import DSI_Capstone_Steemit
from DSI_Capstone_Steemit.utils.utils import(
    load_data_and_description,
    load_raw_cleaned
)
import networkx as nx
# load_data_and_description(data_type = 'tfidf')
data_directory = '../data/'

%matplotlib inline

import seaborn as sns
import community
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import community_louvain as community
output_directory = os.path.join(data_directory,'networkx_votes')




In [ ]:
raw_posts = load_raw_cleaned()
# df_network_x = pd.read_csv('../data/voting_network.csv')
authors = pd.read_csv(os.path.join(data_directory,'accounts.csv'))

In [ ]:
# df_network_x['weight'] = 1/df_network_x['weight']



In [ ]:
# new_string = df_network_x['author'] + ',' + df_network_x['voter']
# new_string = new_string.str.split(',').apply(np.sort)
# df_network_x['author'] = new_string.str.get(0)
# df_network_x['voter'] = new_string.str.get(1)
# df_network_x = df_network_x.groupby(['author','voter']).sum().reset_index()
# df_network_x.to_csv(os.path.join(output_directory,'votes.csv'))

## Shape goes from 5045139 to 4621422 which shows that the differences is how many times  groups have voted for eachother

In [ ]:
# G_fb = nx.from_pandas_dataframe(df = df_network_x,source = 'author',
#                              target='voter',edge_attr= 'weight',create_using = nx.DiGraph())

# joblib.dump(G_fb,os.path.join(output_directory,'directed_graph'))

G_fb = joblib.load(os.path.join(output_directory,'directed_graph'))

In [ ]:
# parts = community.best_partition(G_fb,weight='weight')
# joblib.dump(parts,os.path.join(data_directory,'parts'))

# parts = joblib.load(os.path.join(output_directory,'parts'))

In [ ]:
print nx.info(G_fb)

In [ ]:
comm_cenetrality = nx.communicability_centrality(G_fb)

### Katz Centrality
* Katz centrality[21] is a generalization of degree centrality. Degree centrality measures the number of direct neighbors, and Katz centrality measures the number of all nodes that can be connected through a path, while the contributions of distant nodes are penalized. Mathematically, it is defined as 

In [ ]:
#

In [ ]:
adj = nx.adjacency_matrix(G_fb)
eigenvalues = np.linalg.eigvals(adj.toarray())
max_eig = max(eigenvalues)
print 1/max_eig

### Katz Centrality

This algorithm it uses the power method to find the eigenvector corresponding to the largest eigenvalue of the adjacency matrix of G. The constant alpha should be strictly less than the inverse of largest eigenvalue of the adjacency matrix for the algorithm to converge. The iteration will stop after max_iter iterations or an error tolerance of number_of_nodes(G)*tol has been reached.



In [ ]:
# # phi = (1+np.sqrt(4991))/2.0
# centrality = nx.katz_centrality_numpy(G_fb,0.01, weight = 'weight')
# centrality = pd.DataFrame(zip(centrality.keys(),centrality.values()))
# centrality.columns = ['user','measure']

## Eigenvalue Centrality
* Eigenvector centrality (also called eigencentrality) is a measure of the influence of a node in a network. It assigns relative scores to all nodes in the network based on the concept that connections to high-scoring nodes contribute more to the score of the node in question than equal connections to low-scoring nodes. Google's PageRank and the Katz centrality are variants of the eigenvector centrality.[19]



In [ ]:
eig_cent = nx.eigenvector_centrality_numpy(G_fb, weight = 'weight')

In [ ]:
joblib_dump(eig_cent,'eig_cent')

### Load Centrality
* The load centrality of a node is the fraction of all shortest paths that pass through that node.



In [ ]:
# load_centrality = nx.load_centrality(G_fb,weight = 'weight')

## Betweeness Centrality

* Betweenness is a centrality measure of a vertex within a graph (there is also edge betweenness, which is not discussed here). Betweenness centrality quantifies the number of times a node acts as a bridge along the shortest path between two other nodes. It was introduced as a measure for quantifying the control of a human on the communication between other humans in a social network by Linton Freeman[17] In his conception, vertices that have a high probability to occur on a randomly chosen shortest path between two randomly chosen vertices have a high betweenness.




In [ ]:
# Took too long
# betweeness_cen = nx.betweenness_centrality(G_fb, weight = 'weight')

In [ ]:
joblib_dump(betweeness_cen,'betweeness_cen')

In [ ]:
def joblib_dump(data,filename):
    joblib.dump(data,os.path.join(output_directory,filename))

# HITS

In [ ]:
hits = nx.hits_scipy(G_fb)
joblib_dump(hits,'hits')

In [ ]:
hubs,authorities = hits

In [ ]:
# df_hits = pd.DataFrame([hubs.values(),authorities.values()]).T
# df_hits.columns = ['Hubs','Authorities']
# sns.regplot(data=df_hits,x = 'Hubs',y = 'Authorities',scatter_kws={"alpha": 0.1},)

### Page Rank
* PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links. It was originally designed as an algorithm to rank web pages.



In [ ]:
prank = nx.pagerank(G_fb,weight = 'weight')

In [ ]:
joblib_dump(prank,'prank')

### Core Number
* Return the core number for each vertex. A k-core is a maximal subgraph that contains nodes of degree k or more. The core number of a node is the largest value k of a k-core containing that node.



In [ ]:
G_fb.remove_edges_from(G_fb.selfloop_edges())
core_k = nx.core_number(G_fb)

In [ ]:
core_k

In [ ]:
joblib_dump(prank,'core_k')

In [ ]:
from scipy.stats import linregress

In [ ]:
# pos=nx.spring_layout(G_fb)
# nx.draw(G_fb,pos=pos,
#     node_size=50, with_labels=False)

# for key in pos.keys():
#     x,y = pos[key]
#     pos[key] = np.array([x,y + .05])



# labels=nx.draw_networkx_labels(G_fb,pos=pos)
# # labels=nx.draw_networkx_labels(G_fb,pos={'steemsports': pos['steemsports']},font_color='blue')
# plt.show()


In [ ]:
centrality['cluster'] = centrality['user'].map(parts)
centrality.to_csv('../data/centrality_lookup/centrality_lookup.csv')

In [ ]:
idx = authors['Cluster'] != 'No Group'
idx2 = centrality['cluster'].isin(authors['Cluster'].value_counts().index[0:21].values)


sns.barplot(x = centrality.ix[idx2,'cluster'], y = centrality.ix[idx2,'measure'], orient = 'v')

In [ ]:
authors['Cluster'] = authors['name'].map(parts)
authors['Cluster'] = authors['Cluster'].fillna('No Group')
# idx = authors['Cluster'] != 'No Group'
# authors.ix[idx,'Cluster'] = authors.ix[idx,'Cluster'].astype(int)



In [ ]:
raw_posts['Cluster'] = raw_posts['author'].map(parts)
raw_posts_small = raw_posts[raw_posts['Cluster'].isin(authors['Cluster'].value_counts().index[0:21].values)]

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (8,8))
# idx = authors['Cluster'] != 'No Group'
idx2 = authors['Cluster'].isin(authors['Cluster'].value_counts().index[0:21].values)

sns.countplot(y = authors.ix[idx2,'Cluster'], ax =ax)

In [ ]:
grouped = raw_posts.groupby('Cluster').sum().reset_index()
raw_posts['count'] = 1
grouped_count = raw_posts.groupby('Cluster').count().reset_index()

In [ ]:
top_20_clusters = authors['Cluster'].value_counts().index[0:21].values
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[:]
sns.barplot(x = 'Cluster',y = 'total_payout_value',
            data = raw_posts_plot)
            # plt.savefig('bar_chart.png')

In [ ]:
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[idx]

idx = grouped_count['Cluster'].isin(top_20_clusters)
bar_plot_data = grouped_count.ix[idx,['Cluster','count']]
bar_plot_data['Cluster'] = bar_plot_data['Cluster'].astype(int)
fig,ax = plt.subplots(1,1,figsize = (10,10))

sns.barplot(x = 'count',y = 'Cluster',data=bar_plot_data, orient='h',ax = ax)

In [ ]:
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[idx]

idx = grouped['Cluster'].isin(top_20_clusters)
bar_plot_data = grouped.ix[idx,['Cluster','total_payout_value']]
bar_plot_data['Cluster'] = bar_plot_data['Cluster'].astype(int)
fig,ax = plt.subplots(1,1,figsize = (10,10))

sns.barplot(x = 'total_payout_value',y = 'Cluster',data=bar_plot_data, orient='h',ax = ax)

In [ ]:
top_20_clusters = authors['Cluster'].value_counts().index[0:21].values
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[:]
sns.barplot(x = 'Cluster',y = 'total_payout_value',
            data = raw_posts_plot)
            # plt.savefig('bar_chart.png')

In [ ]:
top_20_clusters = authors['Cluster'].value_counts().index[0:21].values
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[:]
sns.barplot(x = 'Cluster',y = 'number of body tags',
            data = raw_posts_plot)
            # plt.savefig('bar_chart.png')
    
plt.title('Cluster with Average Number of Body Tags')

In [ ]:
top_20_clusters = authors['Cluster'].value_counts().index[0:21].values
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[:]
sns.barplot(x = 'Cluster',y = 'number of body urls',
            data = raw_posts_plot)
            # plt.savefig('bar_chart.png')
    
plt.title('Cluster with Average Number of Body URls')

In [ ]:
top_20_clusters = authors['Cluster'].value_counts().index[0:21].values
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[:]
sns.barplot(x = 'Cluster',y = 'number of youtube urls',
            data = raw_posts_plot)
            # plt.savefig('bar_chart.png')
    
plt.title('Cluster with Average Number of Video URls')

In [ ]:
top_20_clusters = authors['Cluster'].value_counts().index[0:21].values
idx = raw_posts['Cluster'].isin(top_20_clusters)
raw_posts_plot = raw_posts[:]
sns.barplot(x = 'Cluster',y = 'number of body tags',
            data = raw_posts_plot)
            # plt.savefig('bar_chart.png')
    
plt.title('Cluster with Average Number of Body Tags')